In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from postBornCrossSpectra import PostBorn_Bispec
import numpy as np
import Cosmology as C
import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class

import copy
import kernels
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')

In [ ]:
plotpath = './CrossPostBorn/results/plots/vanessa/'

In [ ]:
zmin = 1e-5
zmax = 1090.

kmin = 1e-4
kmax = 50

cosmo = C.Planck2015

a     = np.linspace(1./(1.+zmin),1./(1.+zmax),500)  
z     = 1./a-1.

data  = C.CosmoData(cosmo[1],z)

L     = np.logspace(1,4,100)

# sampling for integrations
phi=np.linspace(0.,2.*np.pi,1000,endpoint=True)
L1=np.logspace(-1,5,400)

LSST_bins  = [0,1,2,3,4,'all']


In [ ]:
#31 term only
def get_31_term_only(L,L1,M):
    return -L**4/np.pi*simps(L1**3*M(L,L1))


In [ ]:
#compute correction following Krause&Hirata

# redefinition with l'-> l-l'
def get_regularized_sum_of_terms_old(L,L1,phi,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2-2.*LL1/LL*nu)
            integrand =LL*LL1**3*(-LL1*nu+LL)*(-LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**2/LL1**4-M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

#always use trapz rule for accurate results!
# redefinition with l'-> l'-l
def get_regularized_sum_of_terms(L,L1,phi,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2+2.*LL1/LL*nu)
            integrand =LL*LL1**3*(LL1*nu+LL)*(LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**2/LL1**4-M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

##functions below are to test accuracy of numerical angular integration
#numerical angular integration
def get_second_sum_of_terms(L,L1,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = LL1*(LL*LL1**2*(-LL1*nu+LL)*(-LL1+LL*nu)**2-LL**2*(LL*LL1*nu)**2)*(M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

#angular integration by hand
def get_second_sum_of_terms_exact(L,L1,M):
    res=[]
    for LL in L:
        integrand = LL**2*L1**5*(M(LL,L1,grid=False)/LL**2/L1**4)
        res+=[simps(integrand,L1)]   
    return 4*np.asarray(res)/np.pi

In [ ]:
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)

Mstarspls = []
PBs =[]
Cls= []
for LSST_bin in LSST_bins:
    second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)
    PB  = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)
    PBs+=[PB]
    Mstarspls+=[PB.Mstarsp]
    Cls+=[PB.CL_born]
ls = PB.ls


In [ ]:
bin_num=5
#these two should be the same
reskk1 = get_regularized_sum_of_terms(L,L1,phi,Mstarspls[bin_num],method='trapz')
reskk1b= get_regularized_sum_of_terms_old(L,L1,phi,Mstarspls[bin_num],method='trapz')
#these two should be the same
reskk2 = get_second_sum_of_terms(L,L1,Mstarspls[bin_num],method='trapz')
reskk2b= get_second_sum_of_terms_exact(L,L1,Mstarspls[bin_num])


In [ ]:
plt.figure()
plt.title('relative difference of two variable transforms')
plt.loglog(L, reskk1b/reskk1-1)
plt.loglog(L, -reskk1b/reskk1+1)
plt.xlabel('$L$',fontsize=20)
plt.show()
plt.savefig(plotpath+'firstIntegral_tests.pdf', bbox_inches='tight')

In [ ]:
# use trapezian rule
lstyles=['-','--','-.']
colors=['blue', 'crimson','green']
i=0
plt.figure()
plt.title('relative difference of analytic and numeric integral')
for method in ['simps','trapz','sum']:
    reskk_ = get_second_sum_of_terms(L,L1,Mstarspls[bin_num],method=method)
    plt.loglog(L, reskk_/reskk2b-1,label=method,ls=lstyles[i],c=colors[i])
    plt.loglog(L, -reskk_/reskk2b+1,label=method,ls=lstyles[i],c=colors[i])
    i+=1
plt.xlabel('$L$',fontsize=20)
plt.legend()
plt.show()
plt.savefig(plotpath+'integral_tests.pdf', bbox_inches='tight')

In [ ]:
plt.figure()
plt.loglog(ls,Cls[bin_num],label='(11) term')
plt.loglog(L, -reskk1,label='integral 1')
plt.loglog(L, reskk2,ls='--',label='integral 2',color='coral')
plt.loglog(L, reskk1+reskk2,ls='--',label='diff',color='crimson')
plt.loglog(L, -(reskk1+reskk2),ls='-',color='crimson')
plt.loglog(L, reskk2b,ls='-.',label='integral 2 exact')
plt.loglog(L, reskk1+reskk2b,ls='-',label='diff 2 ',color='cyan')
plt.loglog(L, -(reskk1+reskk2b),ls='--',color='cyan')
plt.xlim(10,10000)
plt.ylim(1e-13,1e-6)
plt.legend(loc=(1.05,0.35))
plt.ylabel('$C_L^{\kappa\kappa}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.savefig(plotpath+'cl_cross_pB_tests%s.pdf'%LSST_bin, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
plt.loglog(ls,Cls[bin_num],label='(11) term')
plt.loglog(L, -reskk1,label='integral 1')
plt.loglog(L, reskk2,ls='--',label='integral 2',color='coral')
plt.loglog(L, reskk1+reskk2,ls='--',label='diff',color='crimson')
plt.loglog(L, -(reskk1+reskk2),ls='-',color='crimson')
plt.loglog(L, reskk2b,ls='-.',label='integral 1 old way')
plt.loglog(L, reskk1b+reskk2b,ls='-',label='diff 2 ',color='cyan')
plt.loglog(L, -(reskk1b+reskk2b),ls='--',color='cyan')
plt.xlim(10,10000)
plt.ylim(1e-13,1e-6)
plt.legend(loc=(1.05,0.35))
plt.ylabel('$C_L^{\kappa\kappa}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.savefig(plotpath+'cl_cross_pB_tests_2%s.pdf'%LSST_bin, bbox_inches='tight')
plt.show()

In [ ]:
# test case: reproduce Antony's results
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)
second_kernel = kernels.CMB_lens(data.chi_cmb,data)


PB_auto = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)

In [ ]:
resauto1= get_regularized_sum_of_terms(L,L1,phi,PB_auto.Mstarsp,method='trapz')


In [ ]:
resauto2= get_second_sum_of_terms_exact(L,L1,PB_auto.Mstarsp)

In [ ]:
Cl31 =  get_31_term_only(L,L1,PB_auto.Mstarsp)

In [ ]:
plt.figure()
plt.loglog(ls,PB_auto.CL_born,label='sum')
plt.loglog(L, -resauto1,label='integral 1')
plt.loglog(L, -Cl31,label='single term')
plt.loglog(L, resauto2,ls='--',label='integral 2',color='coral')
plt.loglog(L, resauto1+resauto2,ls='--',label='diff',color='crimson')
plt.loglog(L, -(resauto1+resauto2),ls='-',color='crimson')
plt.xlim(10,10000)
plt.ylim(1e-13,1e-6)
plt.legend(loc=(1.05,0.35))
plt.ylabel('$C_L^{\kappa\kappa}$',fontsize=20)
plt.xlabel('$L$',fontsize=20)
plt.savefig(plotpath+'cl_auto_pB_tests%s.pdf'%LSST_bin, bbox_inches='tight')
plt.show()